# Reviewing Automated Machine Learning Explanations

As machine learning becomes more and more and more prevelant, the predictions made by models have greater influence over many aspects of our society. For example, machine learning models are an increasingly significant factor in how banks decide to grant loans or doctors prioritise treatments. The ability to interpret and explain models is increasingly important, so that the rationale for the predictions made by machine learning models can be explained and justified, and any inadvertant bias in the model can be identified.

When you use automated machine learning to train a model, you have the option to generate explanations of feature importance that quantify the extent to which each feature (including engineered features that were generated by preprocessing transformations) influences label prediction. In this lab, you'll explore the explanations generated by an automated machine learning experiment.

## Connect to Your Workspace

The first thing you need to do is to connect to your workspace using the Azure ML SDK.

> **Note**: If the authenticated session with your Azure subscription has expired since you completed the previous exercise, you'll be prompted to reauthenticate.

In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.5.0 to work with azure-ml-demo


## Run an Automated Machine Learning Experiment

To reduce time in this lab, you'll run an automated machine learning experiment on local compute with only three iterations.

Note that the **model_explainability** configuration option is set to **True**.

In [2]:
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Dataset

# Prepare data for training
default_ds = ws.get_default_datastore()
if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')
train_data = ws.datasets.get("diabetes dataset")

# Get compute for training
cluster_name = "aml-cluster"
try:
    # Get the cluster if it exists
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2', max_nodes=2)
    training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
training_cluster.wait_for_completion(show_output=True)

# Configure Auto ML
automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = train_data,
                             n_cross_validations = 2,
                             label_column_name='Diabetic',
                             iterations=3,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=3,
                             featurization='off',
                             model_explainability=True # Generate feature importance!
                             )

# Run the Auto ML experiment
print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'diabetes_automl')
automl_run = automl_experiment.submit(automl_config)
automl_run.wait_for_completion(show_output=True)
RunDetails(automl_run).show()

Dataset already registered.
Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Submitting Auto ML experiment...
Running on remote or ADB.

Current status: DatasetCrossValidationSplit. Generating CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of th

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## View Feature Importance

When the experiment has completed in the widget above, click the run that produced the best result to see its details. Then scroll to the bottom of the visualizations to see the relative feature importance.

You can also view feature importance for the best model produced by the experiment by using the **ExplanationClient** class:

In [5]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient
from azureml.core.run import Run

# Wait for the best model explanation run to complete
model_explainability_run_id = automl_run.get_properties().get('ModelExplainRunId')
print(model_explainability_run_id)
if model_explainability_run_id is not None:
    model_explainability_run = Run(experiment=automl_experiment, run_id=model_explainability_run_id)
    model_explainability_run.wait_for_completion(show_output=True)

# Get the best model (2nd item in outputs)
best_run, fitted_model = automl_run.get_output()

# Get the feature explanations
client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# Overall feature importance
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

AutoML_a670dfdf-3b57-41c7-8c4e-cacde3441dbf_ModelExplain
RunId: AutoML_a670dfdf-3b57-41c7-8c4e-cacde3441dbf_ModelExplain
Web View: https://ml.azure.com/experiments/diabetes_automl/runs/AutoML_a670dfdf-3b57-41c7-8c4e-cacde3441dbf_ModelExplain?wsid=/subscriptions/2a43b2d1-03c5-46d9-aa44-98475cf19902/resourcegroups/azure-ml-demo/workspaces/azure-ml-demo

Execution Summary
RunId: AutoML_a670dfdf-3b57-41c7-8c4e-cacde3441dbf_ModelExplain
Web View: https://ml.azure.com/experiments/diabetes_automl/runs/AutoML_a670dfdf-3b57-41c7-8c4e-cacde3441dbf_ModelExplain?wsid=/subscriptions/2a43b2d1-03c5-46d9-aa44-98475cf19902/resourcegroups/azure-ml-demo/workspaces/azure-ml-demo

Warnings:
This compute target type doesn't support non-Docker runs; overriding run configuration to enable Docker.
Please enable Docker in the environment section of your run configuration to stop seeing this warning message.


Feature	Importance
Pregnancies 	 1.833703915929599
Age 	 0.7376374808097527
BMI 	 0.7250787763459128
Se

## View the Model Explanation in Azure Machine Learning studio

With the experiment run completed, click the link in the widget to see the run in Azure Machine Learning studio, and view the **Explanations** tab. Then:

1. Select the explainer that was created by the automated machine learning run.
2. View the **Global Importance** chart, which shows the overall global feature importance.
3. View the **Summary Importance** chart, which shows each data point from the test data in a *swarm*, *violin*, or *box* plot.
4. Select an individual point to see the **Local Feature Importance** for the individual prediction for the selected data point.

## View Importance for Auto-Engineered Features

Automated machine learning includes the option to try preprocessing the data, which often performs *feature engieering* to create new, derived features from which the model is trained. Let's enable that option and re-run the automated machine learning experiment.

In [6]:
# Configure Auto ML
automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = train_data,
                             n_cross_validations = 2,
                             label_column_name='Diabetic',
                             iterations=3,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=3,
                             featurization='auto', # Enable feature engineering
                             model_explainability=True # Generate feature importance
                             )

# Run the Auto ML experiment
print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'diabetes_automl')
automl_run = automl_experiment.submit(automl_config)
automl_run.wait_for_completion(show_output=True)
RunDetails(automl_run).show()

Submitting Auto ML experiment...
Running on remote or ADB.

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardi

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Featurization is accomplished through the use of [Scikit-Learn transformation pipelines](https://scikit-learn.org/stable/modules/compose.html#combining-estimators) (not to be confused with Azure Machine Learning pipelines!). These produce models that include steps to transform the data before inferencing.

Run the following code to view the steps in the model pipeline, and the importance of the engineered features.

In [10]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient
from azureml.core.run import Run

# Wait for the best model explanation run to complete
model_explainability_run_id = automl_run.get_properties().get('ModelExplainRunId')
print(model_explainability_run_id)
if model_explainability_run_id is not None:
    model_explainability_run = Run(experiment=automl_experiment, run_id=model_explainability_run_id)
    model_explainability_run.wait_for_completion(show_output=True)

# Get the best model (2nd item in outputs)
best_run, fitted_model = automl_run.get_output()

print('Model pipeline steps:')
for step in fitted_model.named_steps:
    print('-',step)
    
# Get the feature explanations
client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation(raw=False)
feature_importances = engineered_explanations.get_feature_importance_dict()

# Overall feature importance
print('\nFeature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

AutoML_328febb0-9527-4fb0-b827-c15595a2daae_ModelExplain
RunId: AutoML_328febb0-9527-4fb0-b827-c15595a2daae_ModelExplain
Web View: https://ml.azure.com/experiments/diabetes_automl/runs/AutoML_328febb0-9527-4fb0-b827-c15595a2daae_ModelExplain?wsid=/subscriptions/2a43b2d1-03c5-46d9-aa44-98475cf19902/resourcegroups/azure-ml-demo/workspaces/azure-ml-demo

Execution Summary
RunId: AutoML_328febb0-9527-4fb0-b827-c15595a2daae_ModelExplain
Web View: https://ml.azure.com/experiments/diabetes_automl/runs/AutoML_328febb0-9527-4fb0-b827-c15595a2daae_ModelExplain?wsid=/subscriptions/2a43b2d1-03c5-46d9-aa44-98475cf19902/resourcegroups/azure-ml-demo/workspaces/azure-ml-demo

Warnings:
This compute target type doesn't support non-Docker runs; overriding run configuration to enable Docker.
Please enable Docker in the environment section of your run configuration to stop seeing this warning message.


Model pipeline steps:
- datatransformer
- MaxAbsScaler
- LightGBMClassifier

Feature	Importance
Pregnan

> **More Information**: For more information Automated machine Learning, see the [Azure ML documentation](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train).